In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import numpy as np
import pandas as pd
import os
from skimage import io
import tensorflow as tf
import keras
from keras import backend as K 
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.layers import GlobalAveragePooling2D, Dense, Activation, BatchNormalization, Dropout, Input
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
import cv2
from keras.applications.vgg16 import  VGG16, preprocess_input
from skimage.util import view_as_windows 
from scipy.io import loadmat
from scipy.stats import pearsonr, spearmanr

#Data loading

In [3]:
!unzip /content/drive/MyDrive/Projet-5-em/Project-kadid10k/PEID.zip

Archive:  /content/drive/MyDrive/Projet-5-em/Project-kadid10k/PEID.zip
   creating: PEID/
  inflating: PEID/data.mat           
   creating: PEID/encimg/
  inflating: PEID/encimg/airport_01_1.j2k  
  inflating: PEID/encimg/airport_01_2.j2k  
  inflating: PEID/encimg/airport_01_3.j2k  
  inflating: PEID/encimg/airport_01_4.j2k  
  inflating: PEID/encimg/airport_01_5.j2k  
  inflating: PEID/encimg/airport_02_1.jpg  
  inflating: PEID/encimg/airport_02_2.jpg  
  inflating: PEID/encimg/airport_02_3.jpg  
  inflating: PEID/encimg/airport_02_4.jpg  
  inflating: PEID/encimg/airport_02_5.jpg  
  inflating: PEID/encimg/airport_02_6.jpg  
  inflating: PEID/encimg/airport_03_1.j2k  
  inflating: PEID/encimg/airport_03_2.j2k  
  inflating: PEID/encimg/airport_03_3.j2k  
  inflating: PEID/encimg/airport_03_4.j2k  
  inflating: PEID/encimg/airport_03_5.j2k  
  inflating: PEID/encimg/airport_03_6.j2k  
  inflating: PEID/encimg/airport_04_1.j2k  
  inflating: PEID/encimg/airport_04_2.j2k  
  inflatin

In [4]:
# Put the reference and degraded images together in the same folder

!cp /content/PEID/refimg/* /content/PEID/encimg/
len(os.listdir('/content/PEID/encimg/'))

1100

In [5]:
train_visual_security_file = pd.read_csv('/content/drive/MyDrive/Projet-5-em/Project-kadid10k/PEID_train_file_visual.csv')
train_visual_security_file.head()

,encrypted_image,reference_image,visual_quality
0,airport.bmp,airport.bmp,1.000000
1,airport_01_1.j2k,airport.bmp,0.023810
2,airport_01_2.j2k,airport.bmp,0.031746
3,airport_01_3.j2k,airport.bmp,0.119048
4,airport_01_5.j2k,airport.bmp,0.547619


In [6]:
train_image_quality_file = pd.read_csv('/content/drive/MyDrive/Projet-5-em/Project-kadid10k/PEID_train_file.csv')
train_image_quality_file.head()

,encrypted_image,reference_image,ground_truth
0,airport.bmp,airport.bmp,1.00
1,airport_01_1.j2k,airport.bmp,0.20
2,airport_01_2.j2k,airport.bmp,0.20
3,airport_01_3.j2k,airport.bmp,0.20
4,airport_01_5.j2k,airport.bmp,0.31


In [7]:
train = train_visual_security_file
train.insert(3, 'ground_truth',train_image_quality_file['ground_truth'].values)
train

,encrypted_image,reference_image,visual_quality,ground_truth
0,airport.bmp,airport.bmp,1.000000,1.00
1,airport_01_1.j2k,airport.bmp,0.023810,0.20
2,airport_01_2.j2k,airport.bmp,0.031746,0.20
3,airport_01_3.j2k,airport.bmp,0.119048,0.20
4,airport_01_5.j2k,airport.bmp,0.547619,0.31
...,...,...,...,...
875,water_09_4.bmp,water.bmp,0.388889,0.22
876,water_09_6.bmp,water.bmp,0.753968,0.44
877,water_10_4.bmp,water.bmp,0.357143,0.26
878,water_10_5.bmp,water.bmp,0.349206,0.27


In [8]:
trainedModel = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

x = GlobalAveragePooling2D()(trainedModel.output)

layer = Dense(512,kernel_initializer = 'normal')(x)
layer = BatchNormalization()(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.2)(layer)

layer = Dense(512, kernel_initializer = 'normal')(layer)
layer = BatchNormalization()(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.2)(layer)

outputs = Dense(2)(layer)
model = Model(inputs = trainedModel.input, outputs =outputs)

58900480/58889256 [==============================] - 1s 0us/step


In [9]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

# Patches extraction and saving to speed up the training process

In [10]:
# os.chdir('/content/PEID/encimg/')
# x, yimage, yvisual = [], [], []
# for encimg, labelVisual, labelImage in zip(train['encrypted_image'], train['visual_quality'], train['ground_truth']):
#   i = 1
#   imageR = image.load_img(encimg)
#   imageR = np.expand_dims(imageR, axis=0)

#   patches = tf.image.extract_patches(images=imageR, sizes=[1, 256, 256, 1], strides=[1, 256,256,1], rates=[1, 1, 1, 1], padding='VALID')
#   patches = tf.reshape(patches,[-1,256,256,3])
#   for patch in patches:
#     reshape_patch = cv2.resize(patch, (224,224), interpolation = cv2.INTER_AREA)
#     test = image.img_to_array(reshape_patch).astype('uint8')
#     filename = str(i)+'_'+encimg.split('.')[0]+'.png'
#     x.append(filename)
#     yimage.append(labelImage)
#     yvisual.append(labelVisual)
#     cv2.imwrite('/content/patches/'+filename, cv2.cvtColor(test, cv2.COLOR_RGB2BGR))
#     i = i + 1

In [11]:
# len(os.listdir('/content/patches/'))

3520

In [12]:
# tempdat = {
#     'images': x,
#     'image_score': yimage,
#     'visual_score':yvisual
# }
# dataset = pd.DataFrame(tempdat, columns=['images','image_score','visual_score'])
# dataset

,images,image_score,visual_score
0,1_airport.png,1.00,1.000000
1,2_airport.png,1.00,1.000000
2,3_airport.png,1.00,1.000000
3,4_airport.png,1.00,1.000000
4,1_airport_01_1.png,0.20,0.023810
...,...,...,...
3515,4_water_10_5.png,0.27,0.349206
3516,1_water_10_6.png,0.35,0.539683
3517,2_water_10_6.png,0.35,0.539683
3518,3_water_10_6.png,0.35,0.539683


In [ ]:
# def train_generator(batch_size):

#   while True:
#     for pos in range(0, len(dataset), batch_size):
#       temp = dataset.iloc[pos:pos + batch_size] 
      
#       encrypted_batch, ytrain = [], []

#       for encimg, y_image, y_visual in zip(temp.images, temp.image_score, temp.visual_score):  
#         encrypted_image = image.load_img('/content/patches/'+encimg)
#         encrypted_image = image.img_to_array(encrypted_image)
   
#         normalise_patch = encrypted_image / 255.0

#         encrypted_batch.append(normalise_patch)
#         ytrain.append([y_image, y_visual])

#       encrypted_batch = np.array(encrypted_batch)
#       ytrain = np.array(ytrain)

#       yield (encrypted_batch, ytrain)

#Data pre-processing
Some necessary pre-processing work has been done
<ul>
  <li>Divide all images into Np small patches of size 256 × 256 without overlapping</li>
  <li>Downsample the Np patches of each image to have a size of 224 × 224
  </li>
  <li>Normalize each image patch by dividing it by 255.0</li>
</ul>

In [13]:
def train_generator(batch_size):

  while True:
    for pos in range(0, len(train), batch_size):
      temp = train.iloc[pos:pos + batch_size] 
      
      encrypted_batch, ytrain = [], []

      for encimg, y_image, y_visual in zip(temp.encrypted_image, temp.ground_truth, temp.visual_quality):  
        encrypted_image = image.load_img('/content/PEID/encimg/'+encimg)
        encrypted_image = image.img_to_array(encrypted_image)

        patches = view_as_windows(encrypted_image,(256, 256, 3),(256, 256, 3)).reshape((-1,256,256,3))
   
        for patch in patches:
          reshape_patch = cv2.resize(patch, (224,224), interpolation = cv2.INTER_AREA)
          normalize_patch = reshape_patch / 255.0

          encrypted_batch.append(normalize_patch)
          ytrain.append([y_image, y_visual])

      encrypted_batch = np.array(encrypted_batch)
      ytrain = np.array(ytrain)

      yield (encrypted_batch, ytrain)

#Model Training

In [15]:
batch_size = 16

for layer in model.layers[:3]:
  layer.trainable = False

for layer in model.layers[3:]:
  layer.trainable = True

model.compile(
    loss= ['mae','mae'],
    loss_weights = [0.4, 0.6],
    optimizer= tf.optimizers.Adam(0.0001)
)

callback = [tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience= 40,
    mode = 'auto',
    min_delta=1e-3,
    restore_best_weights = True,
    verbose =1
)]

history = model.fit(
    train_generator(batch_size),
    epochs = 500,
    verbose = 1,
    batch_size = batch_size,
    callbacks = callback,
    steps_per_epoch= len(train)//batch_size
)

Epoch 1/5
55/55 [==============================] - 69s 1s/step - loss: 0.1416
Epoch 2/5
55/55 [==============================] - 67s 1s/step - loss: 0.1389
Epoch 3/5
55/55 [==============================] - 67s 1s/step - loss: 0.1347
Epoch 4/5
55/55 [==============================] - 67s 1s/step - loss: 0.1349
Epoch 5/5
55/55 [==============================] - 67s 1s/step - loss: 0.1463


In [16]:
model.save('The path of the folder where you want to save your trained model/VGG160406.h5')